In [ ]:
from selenium import webdriver, Timeout
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from datetime import datetime, date
import psycopg2
from psycopg2.extras import RealDictCursor
import os
import json
from populate_events_functions import start_selenium, scrape_venue_html, scrape_venue_json_ld, validate_artist, parse_missed_artists_batch
import html
import pandas as pd
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
from fuzzywuzzy import fuzz
import random


# SCRAPE VENUES

In [6]:
conn = psycopg2.connect(os.getenv('DATABASE_URL_UNPOOLED'))

cur = conn.cursor()

# Get all active venues
cur.execute("""
    SELECT * 
    FROM venues 
    WHERE is_active = TRUE
""")

column_names = [desc[0] for desc in cur.description]
res = cur.fetchall()
venues = [dict(zip(column_names, v)) for v in res]
print('VENUES')
for venue in venues:
    print('       ', venue['name'])

VENUES
        The Chapel
        The Great American Music Hall
        Washington's
        The Independent
        Rickshaw Stop
        The Warfield
        Neck of the Woods
        The Aggie Theatre
        Ace of Spades
        Avogardro's Number
        The Armory
        Cafe du Nord & Swedish American Hall
        Channel 24
        The Mishawaka 
        Goldfield Trading Post


In [7]:
venues

[{'venue_id': 7,
  'name': 'The Chapel',
  'address': '777 Valencia St',
  'city': 'San Francisco',
  'url': 'https://thechapelsf.com/music/?list1page=1',
  'scraping_config': {'filters': {'check_cancelled': False},
   'base_url': 'https://thechapelsf.com/music/?list1page=1',
   'selectors': {'date': 'p.fs-18.bold.mt-1r.date',
    'genre': 'p.fs-12.genre',
    'artist': 'p.fs-12.headliners',
    'event_container': 'div.event-info-block',
    'cancellation_indicator': None},
   'pagination': {'pages': 2,
    'enabled': True,
    'url_pattern': 'https://thechapelsf.com/music/?list1page={page}'},
   'date_format': '%a %b %d'},
  'is_active': True,
  'created_at': datetime.datetime(2025, 9, 23, 0, 17, 45, 698824),
  'updated_at': datetime.datetime(2025, 9, 23, 0, 17, 45, 698824),
  'validation_config': {}},
 {'venue_id': 11,
  'name': 'The Great American Music Hall',
  'address': '859 O’Farrell St.',
  'city': 'San Francisco',
  'url': 'https://gamh.com/',
  'scraping_config': {'base_url':

In [9]:
raw_events =[]
driver = start_selenium()
for target_venue in venues:
    scraping_config = target_venue.get('scraping_config', {})
    pagination = scraping_config.get('pagination', {})
    selectors = scraping_config.get('selectors', {})
    id = target_venue.get('venue_id')
    print(target_venue.get('name'))
    events_list=[]

    soups = []

    if pagination.get('enabled'):
        url_pattern = pagination.get('url_pattern')
        if pagination.get('enabled'): # bool
            pages = pagination.get('pages', 1)
            url_pattern = pagination.get('url_pattern')
            for i in range(1, 6):
                page_url = url_pattern.format(page=i)
                print(f"  → Getting page {i}: {page_url}")
                try:
                    driver.get(page_url)
                    time.sleep(1)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    events = scrape_venue_html(soup, id, target_venue['scraping_config'])
                    raw_events.extend(events)
                    print('added ', len(events))

                except Exception as e:
                    print(f"  ⚠️  Timeout/error loading page {i}: {e}")
                    break
                except TimeoutException:
                    print(f"  ⚠️  Timeout loading {target_venue.get('name')}")
                    continue  # Skip this venue and move to next

    else:
        driver.get(scraping_config.get('base_url'))
        time.sleep(1)
        method = scraping_config.get('scraping_method', 'html')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        if method == 'html':
            events = scrape_venue_html(soup, id, scraping_config)
            raw_events.extend(events)
        elif method == 'json-ld':
            events = scrape_venue_json_ld(soup, id, scraping_config)
            raw_events.extend(events)
        print('added ', len(events))

driver.quit()

The Chapel
  → Getting page 1: https://thechapelsf.com/music/?list1page=1
added  15
  → Getting page 2: https://thechapelsf.com/music/?list1page=2
added  15
  → Getting page 3: https://thechapelsf.com/music/?list1page=3
added  15
  → Getting page 4: https://thechapelsf.com/music/?list1page=4
added  15
  → Getting page 5: https://thechapelsf.com/music/?list1page=5
added  15
The Great American Music Hall
added  11
Washington's
  ⚠️ Error parsing event: time data 'Tuesday, January 13, 2026 @ 8:00 pm 2025' does not match format '%A, %B %d @ %I:%M %p %Y'
  ⚠️ Error parsing event: time data 'Thursday, January 15, 2026 @ 8:00 pm 2025' does not match format '%A, %B %d @ %I:%M %p %Y'
  ⚠️ Error parsing event: time data 'Thursday, January 22, 2026 @ 8:00 pm 2025' does not match format '%A, %B %d @ %I:%M %p %Y'
  ⚠️ Error parsing event: time data 'Friday, January 23, 2026 @ 7:30 pm 2025' does not match format '%A, %B %d @ %I:%M %p %Y'
  ⚠️ Error parsing event: time data 'Saturday, January 24, 202

In [10]:
raw_df = pd.DataFrame(raw_events).drop_duplicates()

In [11]:
venue_id_name_dict = {venue['venue_id']: venue['name'] for venue in venues}
raw_df['venue_name'] = raw_df['venue_id'].apply(lambda x: venue_id_name_dict[x])


In [ ]:
raw_df['venue_name'].value_counts()
raw_df.to_csv('raw_df.csv')


venue_name
The Independent                         72
The Chapel                              50
Rickshaw Stop                           44
The Aggie Theatre                       29
Avogardro's Number                      25
Goldfield Trading Post                  20
The Warfield                            19
Neck of the Woods                       18
The Great American Music Hall           10
The Mishawaka                            9
Cafe du Nord & Swedish American Hall     7
Channel 24                               4
Washington's                             3
The Armory                               1
Name: count, dtype: int64

# VALIDATION


In [ ]:
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
refresh_token = os.getenv('REFRESH_TOKEN')
OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')
current_date = datetime.now().date()
cutoff_in_days=21
auth_manager = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri="http://127.0.0.1:8080",
    scope="playlist-modify-public playlist-modify-private"
)
token_info = auth_manager.refresh_access_token(refresh_token)
sp = spotipy.Spotify(auth=token_info['access_token'])

In [ ]:
from populate_events_functions import validate_artists_parallel

raw_event_names = raw_df['raw_event_name'].values

validated_artists_list = validate_artists_parallel(sp, raw_event_names)

validated_artists_list


🔍 Validating 347 artists with 4 parallel workers...
Private Event could not be found on spotify
Chuck Prophet & His Cumbia Shoes could not be found on spotify
The Mystery Lights, Levitation Room could not be found on spotify
Nora Brown, Stephanie Coleman could not be found on spotify
Ryley Walker, Papa M could not be found on spotify
AZ/DZ name did not match spotify --> Dzidzor Azaglo
ILLEAGLES name did not match spotify --> Balaban and the Bald Illeagles
Soul Brass Band name did not match spotify --> The Soul Rebels
Fleetwood Mask could not be found on spotify
Temptation: A Tribute to New Order could not be found on spotify
Wand name did not match spotify --> Walter Wanderley
Tyler Ramsey, Carl Broemel could not be found on spotify
Cobweb Cabaret could not be found on spotify
The Sun Ra Arkestra name did not match spotify --> Sun Ra And The Arkestra
The Sun Ra Arkestra name did not match spotify --> Sun Ra And The Arkestra
Malibu name did not match spotify --> Malibu Ken
The Sam Chase

Best of San Francisco Stand-up Comedy could not be found on spotify
ESC/ Aggravated Assault/ SHRØUD/ Delve could not be found on spotify
Ashes At Last/ Fall Of The King/ Execution/ Amulet/ Sovereign Suicide could not be found on spotify
Salsa Crazy Mondays could not be found on spotify


Hardcore 4 The Holidays could not be found on spotify
Rumours - A Fleetwood Mac Tribute could not be found on spotify
Daft Disko name did not match spotify --> Daft Funk
Switchman Sleepin' could not be found on spotify
Jammy Buffet - Tribute to Jimmy Buffet could not be found on spotify
UpDating could not be found on spotify
Dead Floyd could not be found on spotify
Jerro name did not match spotify --> Jerron Paxton


Young & Dead name did not match spotify --> Dead Young Poets
CATTLE DECAPITATION NO FEAR FOR TOMORROW NORTH AMERICAN TOUR 2025 could not be found on spotify
Santa Rave: A Holiday Dance Party 18+ could not be found on spotify
Emo Nite 18+ could not be found on spotify
Electric Feels: Indie Rock + Electronic Dance Party (18+) could not be found on spotify
GoldFord: Space of the Heart Tour could not be found on spotify
United We Dance: The Ultimate Rave Experience 18+ could not be found on spotify
Justin Bieber Night: Presented by Stan Society 18+ could not be found on spotify
Priscilla Block  - Things You Didn't See Tour could not be found on spotify
Elevation Rhythm - The Goodbye Yesterday Tour could not be found on spotify
Gasolina Reggaetón Party 18+ could not be found on spotify
ERNEST: LIVE FROM THE SOUTH TOUR could not be found on spotify
Big Bubble Rave 18+ could not be found on spotify
Jacquees with Jaz Karis - California Love Tour could not be found on spotify
Neko Case - Neon G

In [ ]:
event_artist_map

{'The Halloween Extravaganza featuring Guerrilla Radio x Sabotage with School of Rock House Band': ['Guerrilla Radio',
  'Sabotage',
  'School of Rock House Band'],
 'Gimme Gimme Disco': ['Gimme Gimme Disco'],
 '88.9 KRFC presents': [],
 'Fentanyl/ Eliminators/ Giallo/ Bloodstained Concrete/ Robber': ['Fentanyl',
  'Eliminators',
  'Giallo',
  'Bloodstained Concrete',
  'Robber'],
 "Jack's Mannequin": ["Jack's Mannequin"],
 'Spring Polar Plunge **AT RIVERSIDE COLORADO**': [],
 'New Year’s Couples Package + Lodging': [],
 'Rush Archives - Tribute to Rush': ['Rush Archives'],
 'Persuasion - Santana Tribute': ['Persuasion'],
 "Switchman Sleepin'": ["Switchman Sleepin'"],
 'Brainstory, É Arenas (of Chicano Batman)': ['Brainstory', 'É Arenas'],
 'Present/ Giveaway/ Losing Strak/ Stranger Than': [],
 'CHILD': ['CHILD'],
 'Bro Short: Dance Through the Decades **MISH INDOOR STAGE**': [],
 'Afrobeats x Reggaeton': [],
 'Hobo Johnson and The Lovemakers': ['Hobo Johnson and The Lovemakers'],
 'Cl

In [ ]:
new_rows = []

for (raw_event_name, cleaned_artists) in event_artist_map.items():
    # Get the original event data to copy venue_id, date info, etc.
    filtered = raw_df[raw_df['raw_event_name'] == raw_event_name]
    if not filtered.empty:  # Check if any matching rows exist
        original_event = filtered.iloc[0]
    else:
        pass
    
    for artist in cleaned_artists:
        # Get the artist from spotify
        results = sp.search(q=f'artist:{artist}', type='artist', limit=3)
        
        # If you find it on spotify
        if results['artists']['items']:
            name = results['artists']['items'][0]['name']
            # And the names match
            if fuzz.ratio(artist.lower(), name.lower()) > 80:
                genres = results['artists']['items'][0]['genres']
                artist_id = results['artists']['items'][0]['id']
                popularity = results['artists']['items'][0]['popularity']
                
                # Create new row
                new_row = {
                    'venue_id': original_event['venue_id'],
                    'raw_event_name': raw_event_name,
                    'raw_date_text': original_event['raw_date_text'],
                    'parsed_date': original_event['parsed_date'],
                    'spotify_artist_id': artist_id,
                    'spotify_artist_name': name,
                    'artist_popularity': popularity,
                    'genres': genres,
                    'is_cancelled': False
                }
                new_rows.append(new_row)
            else:
                print(f"{artist} name did not match spotify --> {name}")
        else:
            print(f"{artist} could not be found on spotify")

# Append new rows to existing DataFrame
missed_events_df = pd.DataFrame(new_rows)
raw_df = pd.concat([raw_df, missed_events_df], ignore_index=True).drop_duplicates(['raw_event_name', 'raw_date_text'])

Guerrilla Radio name did not match spotify --> Radio Guerrilla
Gimme Gimme Disco could not be found on spotify
Giallo name did not match spotify --> Giallo Point
Bloodstained Concrete could not be found on spotify
Robber name did not match spotify --> Daylight Robbery!
Jack's Mannequin name did not match spotify --> Peter Jackson ft. Die Mannequin and Shadow Bloc
Persuasion name did not match spotify --> The Persuasions
Switchman Sleepin' could not be found on spotify
CHILD name did not match spotify --> Tyler Childers
Hobo Johnson and The Lovemakers could not be found on spotify
ORIHUELA name did not match spotify --> Orihuela M.S.S.
ADULT. name did not match spotify --> Adult Mom
Melvin Seals & JGB could not be found on spotify
AZ name did not match spotify --> Tito Lio Artista
DZ name did not match spotify --> Trundles artist development
The Swashbuckling Doctors could not be found on spotify
Sleepless Nights XIX could not be found on spotify
Aunt Lion could not be found on spotify


In [ ]:
clean_df = raw_df[~raw_df['spotify_artist_id'].isna()]

In [ ]:
conn = psycopg2.connect(os.getenv('DATABASE_URL_UNPOOLED'))

cur = conn.cursor()

# Upsert query - updates if conflict on unique constraint
insert_query = """
INSERT INTO artist_events (
    venue_id,
    raw_event_name,
    raw_date_text,
    genre,
    is_cancelled,
    spotify_artist_name,
    spotify_artist_id,
    artist_popularity,
    event_date
) VALUES %s
ON CONFLICT (venue_id, spotify_artist_name, event_date) 
DO UPDATE SET
    raw_event_name = EXCLUDED.raw_event_name,
    raw_date_text = EXCLUDED.raw_date_text,
    genre = EXCLUDED.genre,
    is_cancelled = EXCLUDED.is_cancelled,
    artist_popularity = EXCLUDED.artist_popularity,
    scraped_at = CURRENT_TIMESTAMP
"""

# Convert DataFrame to list of tuples
data_tuples = []
for _, row in clean_df.iterrows():
    data_tuple = (
        int(row['venue_id']),
        row['raw_event_name'],
        row['raw_date_text'],
        ','.join(row['genres']) if (isinstance(row['genres'], list) and len(row['genres']) > 0) else None,
        bool(row['is_cancelled']),
        row['spotify_artist_name'] if pd.notna(row['spotify_artist_name']) else None,
        row['spotify_artist_id'] if pd.notna(row['spotify_artist_id']) else None,
        int(row['artist_popularity']) if pd.notna(row['artist_popularity']) else None,
        row['parsed_date'] if pd.notna(row['parsed_date']) else None
    )
    data_tuples.append(data_tuple)

# Execute bulk upsert
psycopg2.extras.execute_values(
    cur,
    insert_query,
    data_tuples,
    template=None,
    page_size=100
)

conn.commit()
print(f"Upserted {len(data_tuples)} rows into artist_events table")

Upserted 179 rows into artist_events table
